<a href="https://colab.research.google.com/github/RafsanJany-44/NLP_Research/blob/master/Social_Media_Data(Youtube_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries

In [ ]:
from googleapiclient.discovery import build
import json
import requests
import time
import pandas as pd
import re
import numpy as np

##<font color='coral'>INPUT URL AND Slicing Video ID

In [ ]:
url_input = "https://www.youtube.com/watch?v=yb2MstCVGQY"
video_id_split =url_input.split('=')
video_id=video_id_split[1]
print(video_id)


##<font color='coral'>Seting `api_key`

-> How?<br>
1. Log in to Google Developers Console. [link](https://console.cloud.google.com/apis/library)<br>
2. Create a new project.<br>
3. On the new project dashboard, click Explore & 4. Enable APIs.<br>
4. In the library, navigate to YouTube Data API v3 under YouTube APIs.<br>
5. Enable the API.<br>
6. Create a credential.<br>
7. A screen will appear with the API key.<br>

In [ ]:
api_key = ''

##<font color='coral'>URL Input

In [ ]:
url = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&key='+api_key+'&part=id,statistics'
response_info=requests.get(url).json()


##<font color='coral'>Comments Count

In [ ]:
# Contains all comments (both top-level comments and replies to those comments).
all_comments = []

# Get the total (sum) of comments the video has:
for comment_count in response_info['items']:
  total = int(comment_count['statistics']['commentCount'])

# Show the total amount of comments for the given video:
print("Total comments for (" + video_id + "): " + str(total))

##<font color='coral'>Storing Comments

In [ ]:
def getAllTopLevelCommentReplies(topCommentId, token):
  """
  Recursive function that retrieves the replies a given comment has.
  """

  replies_response=youtube.comments().list(part='snippet',maxResults=100,parentId=topCommentId,pageToken=token).execute()

  for indx, reply in enumerate(replies_response['items']):
    all_comments.append(reply['snippet']['textDisplay'])

  if "nextPageToken" in replies_response:
    return getAllTopLevelCommentReplies(topCommentId, replies_response['nextPageToken'])
  else:
    return []

def get_comments(youtube, video_id, token):
  """
  Recursive function that retrieves the comments (top-level ones) a given video has.
  """

  global all_comments
  totalReplyCount = 0
  token_reply = None

  if (len(token.strip()) == 0):
    all_comments = []

  if (token == ''):
    video_response=youtube.commentThreads().list(part='snippet',maxResults=100,videoId=video_id,order='relevance').execute()
  else:
    video_response=youtube.commentThreads().list(part='snippet',maxResults=100,videoId=video_id,order='relevance',pageToken=token).execute()

  ## ---- for indx, item in enumerate(video_response['items']):
  ## ----   all_comments.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
  ## ----   totalReplyCount = item['snippet']['totalReplyCount']
  ## ----   if (totalReplyCount > 0):
  ## ----     all_comments.append(getAllTopLevelCommentReplies(item['id'], None))
  ## ----
  ## ---- if "nextPageToken" in video_response:
  ## ----   return get_comments(youtube, video_id, video_response['nextPageToken'])
  ## ---- else:
  ## ----   # Remove empty elements added to the list "due to the return in both functions":
  ## ----   all_comments = [x for x in all_comments if len(x) > 0]
  ## ----   return []

  # INICIO

  # Loop comments from the video:
  for indx, item in enumerate(video_response['items']):
    # Append coments:
    all_comments.append("COMMENT WITH " + str(item['snippet']['totalReplyCount']) + " replies: " + item['snippet']['topLevelComment']['snippet']['textDisplay'])

    # Get total reply count:
    totalReplyCount = item['snippet']['totalReplyCount']

    # If the comment has replies, get them:
    if (totalReplyCount > 0):
      # Get replies - first batch:
      replies_response=youtube.comments().list(part='snippet',maxResults=100,parentId=item['id']).execute()
      for indx, reply in enumerate(replies_response['items']):
        # Append the replies to the main array:
        all_comments.append((" "*2) + "=>FIRST CALLBACK REPLY: " + reply['snippet']['textDisplay'])

      # If the reply has a token for get more replies, loop those replies
      # and add those replies to the main array:
      while "nextPageToken" in replies_response:
        token_reply = replies_response['nextPageToken']
        replies_response=youtube.comments().list(part='snippet',maxResults=100,parentId=item['id'],pageToken=token_reply).execute()
        for indx, reply in enumerate(replies_response['items']):
          all_comments.append((" "*4) + "==>WHILE GETTING REPLIES: " + reply['snippet']['textDisplay'])

  # Check if the video_response has more comments:
  if "nextPageToken" in video_response:
    return get_comments(youtube, video_id, video_response['nextPageToken'])
  else:
    # Remove empty elements added to the list "due to the return in both functions":
    all_comments = [x for x in all_comments if len(x) > 0]
    print("Fin")
    return []

  # FIN

  ## 28/04/2022: Reviewed again and here are my results:
  ## Both codes works fine = problem is: youtube does not match the count shown
  ## in the website -comment section- and youtube data api. some comments *might*
  ## be under approval or youtube just removes without updating the "totalReplyCount" value.
  ## also, commment replies are not in date order, those are retrieved in descendent order
  ## (newest replies are first in the results).

all_comments=[]
qtyReplies = 0
qtyMainComments = 0

youtube = build('youtube', 'v3',developerKey=api_key)
comments = get_comments(youtube,video_id,'')

# Show results:
print("All total comments obtained: "  + str(len(all_comments)))

In [ ]:
all_comments

###<font color='coral'>To pandas dataframe

In [ ]:
df = pd.DataFrame(all_comments,columns=['Comments'])
print(df)

#<font color='coral'>Data Refining

##<font color='coral'>Removing EMOJI's

In [ ]:
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
df

##<font color='coral'>Removing URLs

In [ ]:
df['Comments'] = df['Comments'].apply(lambda x: re.split('<a href="https:\/\/.*', str(x))[0])
df

In [ ]:
df['Comments'] = df['Comments'].str.lower()
df